## Imports


In [174]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.tools import DuckDuckGoSearchRun

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain_openai import OpenAIEmbeddings

from langchain.tools.retriever import create_retriever_tool

from langchain.utils.math import cosine_similarity
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from openai import OpenAI
from dotenv import load_dotenv


## Pegando .env api

In [175]:
load_dotenv() 
client = OpenAI()

## loadando modelo

In [176]:

model = ChatOpenAI(
    model='gpt-3.5-turbo-1106',
    temperature=0.7
)

## loaders e splitting

In [177]:

loader_web = WebBaseLoader("https://www.metro.sp.gov.br/")
docs = loader_web.load()
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

## definindo db

In [178]:
embedding = OpenAIEmbeddings()
vectorStore = FAISS.from_documents(splitter.split_documents(docs), embedding=OpenAIEmbeddings())
retriever = vectorStore.as_retriever(search_kwargs={"k": 3})

## Templates

In [179]:
agente_rotas_template = """
Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
Você é um especialista no sistema de Metrô de São Paulo, com foco em determinar as melhores rotas para os passageiros. \
      Você tem amplo conhecimento sobre as linhas do metrô, suas interligações e horários de pico, fornecendo alternativas rápidas e eficientes para quem utiliza o metrô. \
      Responda em ate 15 palavras, com informações claras e precisas.
"""

agente_horarios_template = """
Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo. \
    Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô. \
        Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço. \
        Responda em ate 15 palavras, com informações claras e precisas.
        

"""

agente_politicas_template = """
Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
Você é um especialista em políticas públicas relacionadas ao sistema de Metrô de São Paulo. \
    Você tem conhecimento detalhado sobre as leis, regulamentos e iniciativas que afetam o metrô, e oferece respostas claras \
        e baseadas em fatos sobre como essas políticas impactam os passageiros. \
        Responda em ate 15 palavras, com informações claras e precisas.
"""



## Exemplos de perguntas pra semantic routing

In [180]:
rotas_perguntas = [
    "Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?",
    "Como posso evitar as estações mais lotadas durante o horário de pico?",
    "Quais são as melhores conexões entre as linhas azul e verde do metrô?",
]

horarios_perguntas = [
    "Há alguma greve no Metrô de São Paulo programada para esta semana?",
    "O metrô está funcionando normalmente durante o feriado?",
    "Quais são os horários de operação do metrô aos domingos?",
]

politicas_perguntas = [
    "Quais são as políticas de acessibilidade nas estações do Metrô de São Paulo?",
    "Como as tarifas do metrô são decididas e ajustadas?",
    "Quais são as políticas de sustentabilidade adotadas pelo Metrô de São Paulo?",
]


## Fazendo embeddings com os exemplos pra calcular qual o cosseno e poder classificar a dúvida

In [181]:
embeddings = OpenAIEmbeddings()

rotas_perguntas_embeddings = embeddings.embed_documents(rotas_perguntas)
horarios_perguntas_embeddings = embeddings.embed_documents(horarios_perguntas)
politicas_perguntas_embeddings = embeddings.embed_documents(politicas_perguntas)

## LLM routing com busca semântica

In [182]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])

    rotas_similarity = cosine_similarity([query_embedding], rotas_perguntas_embeddings)[0]

    horarios_similarity = cosine_similarity([query_embedding], horarios_perguntas_embeddings)[0]

    politicas_similarity = cosine_similarity([query_embedding], politicas_perguntas_embeddings)[0]

    max_similarity = max(
        max(rotas_similarity), max(horarios_similarity), max(politicas_similarity)
    )

    if max_similarity == max(rotas_similarity):
        prompt_escolhido = "rotas"
    elif max_similarity == max(horarios_similarity):
        prompt_escolhido = "horarios"
    else:
        prompt_escolhido = "politicas"
    return prompt_escolhido

## Tool do duck duck go

In [183]:
duckduckgo = DuckDuckGoSearchRun()

api_wrapper = WikipediaAPIWrapper(top_k_results=1,document_content_chars_max=200)   
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)

retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Pesquisa e retorna informações sobre o Metrô de São Paulo.")
tools = [duckduckgo, retriever_tool, wiki]

## gerando prmpt a depender do output do routing

In [184]:
def gerar_prompt(prompt_escolhido):
    templates = {
        "rotas": agente_rotas_template,
        "horarios": agente_horarios_template,
        "default": agente_politicas_template
    }

    template = templates.get(prompt_escolhido, templates["default"])

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    return prompt


## Definindo pergunta

In [185]:
pergunta = str(input("Digite sua pergunta: "))

In [186]:
def gerar_prompt(prompt_escolhido):
    templates = {
        "rotas": agente_rotas_template,
        "horarios": agente_horarios_template,
        "default": agente_politicas_template
    }

    template = templates.get(prompt_escolhido, templates["default"])

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    return prompt


In [187]:
prompt = gerar_prompt(prompt_router({"query": pergunta}))

agent = create_openai_tools_agent(model, tools, prompt)
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer.invoke({"input":pergunta})





> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'como vou da linha verde para a linha amarela no metrô de são paulo'}`


Para onde você vai?Saiba como chegar ao seu destino






Direto do Metrô

Metrô | Cia. do Metropolitano de São Paulo





























 





Skip to main content











SP + Digital







/governosp













SP + Digital







/governosp

Imóveis para venda e locação








Anuncie no Metrô






Serviços de Consultoria






Comércio






Parcerias Institucionais






Expansão e Obras





 


Obras da Linha 2-VerdePara ir da Linha Verde para a Linha Amarela, pegue a Linha Verde até a estação Paulista e então troque para a Linha Amarela.

> Finished chain.


{'input': 'como vou da linha verde pra amarela?',
 'output': 'Para ir da Linha Verde para a Linha Amarela, pegue a Linha Verde até a estação Paulista e então troque para a Linha Amarela.'}